In [1]:
import numpy as np

def payoff_density_integrand(x, S0, K, r, q, sigma, T):
    if x <= 0 or x >= K:
        return 0.0
    mu_ln = np.log(S0) + (r - q - 0.5 * sigma**2) * T
    sigma_ln = sigma * np.sqrt(T)
    log_term = np.log(x) - mu_ln
    density = (1 / (x * sigma_ln * np.sqrt(2 * np.pi))) * np.exp(-0.5 * (log_term / sigma_ln) ** 2)
    return np.sqrt(K - x) * density

def simpsons_rule(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = np.array([f(xi) for xi in x])
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Parameters
S0 = 50
K = 50
q = 0.02
r = 0.04
sigma = 0.3
T = 0.25
discount = np.exp(-r * T)

# Function to evaluate
f = lambda x: payoff_density_integrand(x, S0, K, r, q, sigma, T)

# Iterate to desired accuracy
tol = 1e-5  # Four decimal digit accuracy
n = 4
prev_val = simpsons_rule(f, 1e-10, K, n)  # start near 0 to avoid log(0)
print(f"n={n}, value={discount * prev_val:.10f}")

while True:
    n *= 2
    curr_val = simpsons_rule(f, 1e-10, K, n)
    diff = abs(curr_val - prev_val)
    print(f"n={n}, value={discount * curr_val:.10f}, diff={diff:.2e}")
    if diff < tol:
        break
    prev_val = curr_val


n=4, value=0.7119117896
n=8, value=1.0783479858, diff=3.70e-01
n=16, value=1.0966648366, diff=1.85e-02
n=32, value=1.1110659580, diff=1.45e-02
n=64, value=1.1163674860, diff=5.35e-03
n=128, value=1.1182622644, diff=1.91e-03
n=256, value=1.1189346413, diff=6.79e-04
n=512, value=1.1191727093, diff=2.40e-04
n=1024, value=1.1192569329, diff=8.51e-05
n=2048, value=1.1192867193, diff=3.01e-05
n=4096, value=1.1192972518, diff=1.06e-05
n=8192, value=1.1193009759, diff=3.76e-06


In [9]:
import numpy as np

def lognormal_pdf_y(y, mu, sigma2):
    """PDF of lognormal variable y = S(T)/S(0)"""
    if y <= 0:
        return 0.0
    return (1 / (y * np.sqrt(2 * np.pi * sigma2))) * np.exp(-((np.log(y) - mu) ** 2) / (2 * sigma2))

def integrand_y(y, S0, K, mu, sigma2):
    """Integrand in terms of y = S(T)/S(0)"""
    if y <= 0 or S0 * y >= K:
        return 0.0
    return np.sqrt(K - S0 * y) * lognormal_pdf_y(y, mu, sigma2)

def simpsons_rule(f, a, b, n):
    """Composite Simpson's Rule"""
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = np.array([f(xi) for xi in x])
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Parameters
S0 = 50
K = 50
q = 0.02
r = 0.04
sigma = 0.3
T = 0.25

mu = (r - q - 0.5 * sigma**2) * T
sigma2 = sigma**2 * T
discount = np.exp(-r * T)

# Integration limit for y
y_upper = K / S0

# Wrapper function
f = lambda y: integrand_y(y, S0, K, mu, sigma2)

# Adaptive Simpson's Rule
tol = 1e-5  # four-digit precision
n = 4
prev_val = simpsons_rule(f, 1e-10, y_upper, n)
print(f"n={n}, value={discount * prev_val:.10f}")

while True:
    n *= 2
    curr_val = simpsons_rule(f, 1e-10, y_upper, n)
    diff = abs(curr_val - prev_val)
    print(f"n={n}, value={discount * curr_val:.10f}, diff={diff:.2e}")
    if diff < tol:
        break
    prev_val = curr_val


n=4, value=0.7119117898
n=8, value=1.0783479858, diff=3.70e-01
n=16, value=1.0966648366, diff=1.85e-02
n=32, value=1.1110659580, diff=1.45e-02
n=64, value=1.1163674860, diff=5.35e-03
n=128, value=1.1182622644, diff=1.91e-03
n=256, value=1.1189346413, diff=6.79e-04
n=512, value=1.1191727093, diff=2.40e-04
n=1024, value=1.1192569329, diff=8.51e-05
n=2048, value=1.1192867193, diff=3.01e-05
n=4096, value=1.1192972518, diff=1.06e-05
n=8192, value=1.1193009759, diff=3.76e-06


In [13]:
def integrand_y(y, S0, K, mu, sigma2):
    """Integrand in terms of y = S(T)/S(0)"""
    if y <= 0 or S0 * y <= K:
        return 0.0
    return np.sqrt(S0 * y-K) * lognormal_pdf_y(y, mu, sigma2)

n = 4
prev_val = simpsons_rule(f, y_upper, 10, n)
print(f"n={n}, value={discount * prev_val:.10f}")

while True:
    n *= 2
    curr_val = simpsons_rule(f, y_upper, 10, n)
    diff = abs(curr_val - prev_val)
    print(f"n={n}, value={discount * curr_val:.10f}, diff={diff:.2e}")
    if diff < tol:
        break
    prev_val = curr_val

n=4, value=0.0000000000
n=8, value=0.0000371141, diff=3.75e-05
n=16, value=0.0707776321, diff=7.15e-02
n=32, value=0.7068094942, diff=6.42e-01
n=64, value=1.0285016239, diff=3.25e-01
n=128, value=1.0815847649, diff=5.36e-02
n=256, value=1.0996099476, diff=1.82e-02
n=512, value=1.1060558143, diff=6.51e-03
n=1024, value=1.1083374659, diff=2.30e-03
n=2048, value=1.1091434674, diff=8.14e-04
n=4096, value=1.1094281988, diff=2.88e-04
n=8192, value=1.1095288149, diff=1.02e-04
n=16384, value=1.1095643780, diff=3.59e-05
n=32768, value=1.1095769496, diff=1.27e-05
n=65536, value=1.1095813940, diff=4.49e-06


In [14]:
import numpy as np

def f_Y(y, mu, sigma2):
    if y <= 0:
        return 0.0
    return (1 / (y * np.sqrt(2 * np.pi * sigma2))) * np.exp(-((np.log(y) - mu) ** 2) / (2 * sigma2))

def integrand_unit(u, S0, K, mu, sigma2):
    if u <= 0 or u >= 1:
        return 0.0
    y = (K / S0) + (1 - u) / u
    sqrt_term = np.sqrt(S0 * (1 - u) / u)
    fy = f_Y(y, mu, sigma2)
    return sqrt_term * fy / u**2

def simpsons_rule(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = np.array([f(xi) for xi in x])
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Parameters
S0 = 50
K = 50
q = 0.02
r = 0.04
sigma = 0.3
T = 0.25

mu = (r - q - 0.5 * sigma**2) * T
sigma2 = sigma**2 * T
discount = np.exp(-r * T)

# Integral over u in (0, 1)
f = lambda u: integrand_unit(u, S0, K, mu, sigma2)

# Adaptive Simpson's Rule
tol = 1e-5
n = 4
prev_val = simpsons_rule(f, 1e-10, 1 - 1e-10, n)
print(f"n={n}, value={discount * prev_val:.10f}")

while True:
    n *= 2
    curr_val = simpsons_rule(f, 1e-10, 1 - 1e-10, n)
    diff = abs(curr_val - prev_val)
    print(f"n={n}, value={discount * curr_val:.10f}, diff={diff:.2e}")
    if diff < tol:
        break
    prev_val = curr_val


n=4, value=0.7006370294
n=8, value=1.0684080316, diff=3.71e-01
n=16, value=1.0869361028, diff=1.87e-02
n=32, value=1.1013463158, diff=1.46e-02
n=64, value=1.1066488366, diff=5.36e-03
n=128, value=1.1085434835, diff=1.91e-03
n=256, value=1.1092156815, diff=6.79e-04
n=512, value=1.1094536398, diff=2.40e-04
n=1024, value=1.1095378048, diff=8.50e-05
n=2048, value=1.1095675613, diff=3.01e-05
n=4096, value=1.1095780788, diff=1.06e-05
n=8192, value=1.1095817953, diff=3.75e-06
